# Python tricks by DAN BADER

## Patterns for cleaner python

In [8]:
def apply_discount(p , d):
    price = -p * (d * 0.01)
    assert 0 < price < 100 , ('This should never happen, but it does ')
    print (price)
apply_discount(100, 10)


AssertionError: This should never happen, but it does 

In [7]:
def apply_discount(p , d):
    price = p * (d * 0.01)
    assert(0 < price < 100)
    print (price)
apply_discount(100, 10)

10.0


### Caveats
* The biggest caveat with using asserts in Python is that assertions can be globally disabled3 with the -O and -OO command line switches, as well as the PYTHONOPTIMIZE environment variable in CPython.

* Note below that the assertion will never fails since tuples are always true
 

In [13]:

def apply_discount(p , d):
    price = p * (d * 0.01)
    assert (1 == 2, 'This should fail')
    print (price)
apply_discount(100, 10)



10.0


<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-13-aa2d2748fbe8>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (1 == 2, 'This should fail')


### <font color="red"> Suggestion - Dont enclose assert in parentheses </font>

## Problem of missing commas

In [14]:
lis = [
    "hello",
    "how",
    "are"
    "you"
]
lis

['hello', 'how', 'areyou']

`This so-called “string literal concatenation” is intentional and documented behavior.`

In [17]:
my_str = (
    "hello sentence 1! \n"
    "hello sentence 2! \n"
    "hello sentence 3!"
)

print (my_str)

hello sentence 1! 
hello sentence 2! 
hello sentence 3!


### <font color="red"> Suggestion - Use comma at the end of all entries as well </font>

## Context manager and if statement 
### Where does with statement help  ?
* It helps in simplifying some common resource management problems

#### <font color="red">Note that this code won't ensure file descriptor closure if there was an exception while writing and can hence cause a file descriptor leak </font>

### Context Manager
* A protocol that our object needs to follow to cater to the with statement
* All it needs is - 
    * \__exit__
    * \__enter\__  method
* Python will automatically call them as per the resource management cycles

In [18]:
class ManagedFile:
    def __init__(self , name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name , w)
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

### Using decorators

In [20]:
from contextlib import contextmanager

@contextmanager
def ManagedFile(name):
    try:
        f = open(name,"w")
        yield f
    finally:
        f.close()

In [27]:
#indentor
class Indenter:
    def __init__(self):
        self.level = 0
        self.toggle = False
        
    def __enter__(self):
        self.level += 1
        return self
        
    def __exit__(self , exc_type , exc_val , exc_tb):
        self.level -= 1
        
    def print(self , text):
        print("\t"*self.level + text)

print ("Hi Vaibhav!")
with Indenter() as indent:
    indent.print("Hey!")
    with indent:
        indent.print("What's up")
        with indent:
            indent.print("I am fine!")
        indent.print("See you later!")
    indent.print("Byeee!")
print ("Chat done!")

Hi Vaibhav!
	Hey!
		What's up
			I am fine!
		See you later!
	Byeee!
Chat done!


In [45]:
#indentor
class Chat:
    def __init__(self):
        self.level = 0
        self.toggle = True
        
    def __enter__(self):
        if self.toggle:
            self.level += 1
        else:
            self.level -= 1
        return self
        
    def __exit__(self , exc_type , exc_val , exc_tb):
        pass
        
    def print(self , text):
        print("\t"*self.level + text)
        self.toggle = not self.toggle

In [46]:
print ("Hey Hi!")
with Chat() as c:
    c.print("Hello")
    with c:
        c.print("How are you")
        with c:
            c.print("I am fine!")
            with c:
                c.print("Ok Take care!")

Hey Hi!
	Hello
How are you
	I am fine!
Ok Take care!


In [72]:
#Code execution time
import time
class Time:
    def __init__(self):
        self.current_time = 0

    def __enter__(self):
        self.current_time = time.time()

    def __exit__(self, exc_time, exc_val, exc_tb):
        print ("Execution time : {}".format(time.time() - self.current_time))
        
with Time() as t:
    for k in range(1500):
        with Time() as t1:    
            p = 1
            for i in range(10):
                p = p * i

Execution time : 2.86102294921875e-06
Execution time : 4.291534423828125e-06
Execution time : 4.0531158447265625e-06
Execution time : 3.0994415283203125e-06
Execution time : 4.0531158447265625e-06
Execution time : 4.0531158447265625e-06
Execution time : 3.814697265625e-06
Execution time : 3.0994415283203125e-06
Execution time : 3.0994415283203125e-06
Execution time : 4.291534423828125e-06
Execution time : 2.86102294921875e-06
Execution time : 3.814697265625e-06
Execution time : 4.0531158447265625e-06
Execution time : 4.0531158447265625e-06
Execution time : 2.86102294921875e-06
Execution time : 3.814697265625e-06
Execution time : 4.0531158447265625e-06
Execution time : 4.0531158447265625e-06
Execution time : 4.0531158447265625e-06
Execution time : 4.0531158447265625e-06
Execution time : 3.814697265625e-06
Execution time : 8.106231689453125e-06
Execution time : 7.152557373046875e-06
Execution time : 5.0067901611328125e-06
Execution time : 3.0994415283203125e-06
Execution time : 4.0531158

In [67]:
#decorators
from contextlib import contextmanager

@contextmanager
def Time():
    try:
        current_time = time.time()
        yield current_time
        
    finally:
        print ("Execution time : {}".format(time.time() - current_time))
        
with Time() as t:
    for k in range(1500):
        with Time() as t1:    
            p = 1
            for i in range(10):
                p = p * i

Execution time : 2.86102294921875e-06
Execution time : 4.291534423828125e-06
Execution time : 2.86102294921875e-06
Execution time : 3.0994415283203125e-06
Execution time : 2.86102294921875e-06
Execution time : 1.9073486328125e-06
Execution time : 2.1457672119140625e-06
Execution time : 1.9073486328125e-06
Execution time : 2.1457672119140625e-06
Execution time : 3.0994415283203125e-06
Execution time : 3.0994415283203125e-06
Execution time : 2.1457672119140625e-06
Execution time : 1.9073486328125e-06
Execution time : 1.9073486328125e-06
Execution time : 3.0994415283203125e-06
Execution time : 2.1457672119140625e-06
Execution time : 2.1457672119140625e-06
Execution time : 2.1457672119140625e-06
Execution time : 3.0994415283203125e-06
Execution time : 1.9073486328125e-06
Execution time : 1.9073486328125e-06
Execution time : 2.1457672119140625e-06
Execution time : 3.0994415283203125e-06
Execution time : 2.1457672119140625e-06
Execution time : 3.0994415283203125e-06
Execution time : 2.145767

## Different _ naming patterns in python
* \_var
* var_
* \_\_var
* \_\_var\_\_
* _

---
### <font color="blue"> Single Leading underscore </font>
* It is generally used as a hint to the programmer
    * The interpreter doesnt restrict anything but the community accept such things as intended for internal use 

* Leading underscore affects the way the names get imported from modules
    * If we try something like
    ```python
    #abc.py
    def a():
        pass
       
    def _b():
        pass
    from abc import *
    _b()
    
    ```
    * This will flag an error

* General recommendation -
> dont import wildcard

***

### <font color="blue"> Single Trailing underscore </font>

In [77]:
def func(class):
    a = class * 10
func()

SyntaxError: invalid syntax (<ipython-input-77-8ff48b2efcc6>, line 1)

In [80]:
def func(class_):
    a = class_ * 10
    
func(10)

`Used to avoid naming conflicts with python keywords`
***

### <font color="blue"> Double Leading underscore (DUNDERS)</font>
`Name Mangling` <br>
A double underscore causes the interpreter to rewrite the attribute name in order to avoid naming conflicts in subclasses

In [82]:
class Parent:
    def __init__(self):
        self.foo = 11
        self.__bar = 22
p = Parent()
dir(p)

['_Parent__bar',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'foo']

`Note that __bar is mangled to '_Parent__bar'`

In [86]:
class Child(Parent):
    def __init__(self):
        super().__init__()
        self.foo = "hello"
        self.__bar = "hi"
c = Child()
dir(c)

['_Child__bar',
 '_Parent__bar',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'foo']

`Note that foo gets updated but bar gets a copy from Parent also`

In [87]:
p.foo

11

In [88]:
p.__bar

AttributeError: 'Parent' object has no attribute '__bar'

In [90]:
p._Parent__bar

22

`Name mangling applies to any name starting with two underscore characters that is used in a class context. <br>
Thus it applies to both variables and methods
`

### <font color="blue"> Double Leading and Double trailing underscore (DUNDERS)</font>
* Such names are left unscathed by the interpreter
* Usually they are used for special tasks like **`__init__ or __call__`**


### <font color="blue"> Single underscore</font>

* Generally used as a temp variable or a placeholder
* Its just a convention
* Saves the result of previous evaluation

In [92]:
for _ in range(3):
    print ("hello")

hello
hello
hello


In [93]:
a = [1,2,3,4]
b, _, _, d = a
d

4

In [96]:
20 + 3

23

In [97]:
_

3

***
## String formatting
### <font color="blue"> New style of formatting</font>


In [107]:
name = "vbs"
hex_val = 12345
print ("Hey {name} , Your no is 0x{hex_:x}".format(name=name, hex_=hex_val))
print ("Hey {} , Your no is 0x{:x}".format(name, hex_val))

Hey vbs , Your no is 0x3039
Hey vbs , Your no is 0x3039


### <font color="blue"> Literal string interpolation - Formatted string literals</font>

In [115]:
f"Hello!{name}, Here's your number - {hex_val:#x}!"

"Hello!vbs, Here's your number - 0x3039!"

In [112]:
f"Two plus two = {2+2} and hex_val * two = {hex_val * 2}"

'Two plus two = 4 and hex_val * two = 24690'